In [ ]:
# -*- coding: utf-8 -*-
import zipfile, io, os, re, time, sys

# === Словарь ИМЕН ТОЛЬКО ДЛЯ 7 КЛАССА (как ты дал) ===
TASK_NAMES = {
    "physics__7.avgspd.zip": {
        "1": "Половинки",
        "2": "Катер",
        "3": "Плывем по течению",
        "4": "Смотрите под ноги",
        "5": "Бег по кругу",
        "6": "График средней скорости от пути",
        "7": "Бездорожье"
    },
    "physics__7.force.zip": {
        "1": "Перетягивание каната"
    },
    "physics__7arhimed.zip": {
        "1": "Шар повесили",
        "2": "Топим пластилином",
        "3": "На границе",
        "4": "Куб не тонет",
        "5": "Поплыли",
        "6": "Ледяные весы"
    },
    "physics__7friction.zip": {
        "1": "Куда везем мы кирпичи",
        "2": "Никак не сдвинуть",
        "3": "Снова ящик",
        "4": "Зажали"
    },
    "physics__7graphs.zip": {
        "Максимальная скорость": "Максимальная скорость",
        "Скорость от пути": "Скорость от пути",
        "Максимальное опережение": "Максимальное опережение",
        "Средняя скорость от пути": "Средняя скорость от пути"
    },
    "physics__7kinblok.zip": {
        "1": "Нить на блоке",
        "2": "Эвакуатор"
    },
    "physics__7mgN.zip": {
        "1": "Вес",
        "2": "Нормальная Реакция",
        "3": "Шары приятягиваются",
        "4": "Луннная гравитация",
        "5": "Солнце и земля"
    },
    "physics__7pressure.zip": {
        "1": "Давление мальчика",
        "2": "Большой столб",
        "3": "Масло и водица",
        "4": "Давить на воду",
        "5": "Намудрили",
        "6": "Пластилин на дне"
    },
    "physics__7rpo.zip": {
        "1": "Почти Феррари",
        "2": "Длинный поезд",
        "3": "График x(t)"
    },
    "physics__7springs.zip": {
        "1": "Растяжка",
        "2": "2 пружины последоваительно",
        "3": "теперь параллельно",
        "4": "2 хорошо, а 3 лучше",
        "5": "Разрезали поровну",
        "6": "Увеличиваем жесткость"
    },
    "physics__7statblock.zip": {
        "Два блока": "Два блока",
        "Змейкой": "Змейкой",
        "Ограниченное равновесие": "Ограниченное равновесие",
        "Лесенка": "Лесенка",
        "Намудрили": "Намудрили"
    },
    "physics__7torque.zip": {
        "1": "Равновесие",
        "2": "Груз на палке",
        "3": "Скизифова улитка",
        "4": "Соломинка",
        "5": "рычаг блок и нить"
    },
    "physics__7turmech.zip": {
        "Потерянная информация": "Потерянная информация",
        "Сила на дно": "Сила на дно",
        "Перемещение груза": "Перемещение груза",
        "Люди на рычагах": "Люди на рычагах",
        "Смещение груза": "Смещение груза"
    },
    "physics__7work.zip": {
        "1": "Работа крана",
        "2": "День на стройке",
        "3": "Полезный блок"
    }
}

SOURCE_DIR = "."
OUTPUT_DIR = "./processed"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# \answerinput{что-угодно} -> \answerinput{}
ANSWERINPUT_RE = re.compile(r'(\\answerinput)\s*\{[^}]*\}')

def clean_tex(content: str) -> str:
    return ANSWERINPUT_RE.sub(r'\1{}', content)

def is_html(name: str) -> bool:
    return name.lower().endswith(".html")

def rename_tex_to_main_zip(single_problem_zip: zipfile.ZipFile, new_zip: zipfile.ZipFile):
    """
    Копирует содержимое задачи, удаляя .html, переименовывая первый .tex -> main.tex
    и чистя \answerinput. Возвращает (new_main_path, orig_tex_basename, html_removed_count, cleaned_flag)
    """
    html_removed = 0
    cleaned_cnt = 0
    found_tex = None  # полный путь к исходному tex
    tex_basename_for_name = None

    # Сначала найдём первый .tex для переименования
    for nm in single_problem_zip.namelist():
        if nm.endswith("/") or is_html(nm):
            continue
        if nm.lower().endswith(".tex") and found_tex is None:
            found_tex = nm
            tex_basename_for_name = os.path.splitext(os.path.basename(nm))[0]

    # Теперь копируем все файлы с преобразованиями
    new_main_path = None
    for nm in single_problem_zip.namelist():
        if nm.endswith("/"):
            # каталоги можно пропустить — zip их создаст автоматически
            continue
        if is_html(nm):
            html_removed += 1
            continue

        data = single_problem_zip.read(nm)

        if nm == found_tex:
            # чистим \answerinput{..}
            try:
                text = data.decode("utf-8", errors="ignore")
            except Exception:
                text = data.decode("utf-8", errors="ignore")
            new_text = clean_tex(text)
            if new_text != text:
                cleaned_cnt += 1
            data = new_text.encode("utf-8")

            # переименовать в main.tex в той же директории
            dname = os.path.dirname(nm)
            new_main_path = os.path.join(dname, "main.tex") if dname else "main.tex"
            new_zip.writestr(new_main_path, data)
        else:
            # остальные файлы копируем как есть
            new_zip.writestr(nm, data)

    return new_main_path, tex_basename_for_name, html_removed, cleaned_cnt

def process_root_zip(root_zip_name: str):
    root_html_removed = 0
    inner_html_removed = 0
    tex_cleaned = 0
    tasks_written = 0

    root_zip_path = os.path.join(SOURCE_DIR, root_zip_name)
    print(f"\n================================================================================")
    print(f"📦 Обработка архива: {root_zip_name}")

    with zipfile.ZipFile(root_zip_path, 'r') as root_zip:
        # найти problems.zip (любой путь внутри zip)
        problems_member = next((nm for nm in root_zip.namelist() if nm.endswith("problems.zip")), None)
        if not problems_member:
            print("  ⚠️ problems.zip не найден — пропуск архива.")
            return (0, 0, 0, 0)

        # читаем исходный problems.zip
        problems_data = io.BytesIO(root_zip.read(problems_member))
        with zipfile.ZipFile(problems_data, 'r') as problems_zip:
            # отберём только задачи-архивы, сохранив порядок
            problem_zips = [p for p in problems_zip.namelist() if p.lower().endswith(".zip")]

            # если вдруг в самом problems.zip лежали .html — не копируем их
            html_in_problems = [p for p in problems_zip.namelist() if is_html(p)]
            inner_html_removed += len(html_in_problems)

            print(f"  • Найдено задач: {len(problem_zips)}")
            if html_in_problems:
                print(f"  • Удалено .html прямо в problems.zip: {len(html_in_problems)}")

            # соберём новый problems.zip
            new_problems_buffer = io.BytesIO()
            with zipfile.ZipFile(new_problems_buffer, 'w', compression=zipfile.ZIP_DEFLATED) as new_problems_zip:
                for idx, p in enumerate(problem_zips, start=1):
                    tasks_written += 1
                    inner_bytes = io.BytesIO(problems_zip.read(p))
                    with zipfile.ZipFile(inner_bytes, 'r') as single_problem_zip:
                        new_problem_buf = io.BytesIO()
                        with zipfile.ZipFile(new_problem_buf, 'w', compression=zipfile.ZIP_DEFLATED) as new_problem_zip:
                            new_main_path, tex_base, html_removed, cleaned_cnt = rename_tex_to_main_zip(
                                single_problem_zip, new_problem_zip
                            )
                            inner_html_removed += html_removed
                            tex_cleaned += cleaned_cnt

                            # имя задачи
                            mapping = TASK_NAMES.get(root_zip_name, {})
                            orig_key = os.path.splitext(os.path.basename(p))[0]
                            # если есть словарь для этого архива — берём оттуда; иначе — из имени .tex; fallback — имя zip
                            if mapping:
                                name_value = mapping.get(orig_key, tex_base or orig_key)
                                name_source = "словарь7" if orig_key in mapping else ("имя .tex" if tex_base else "имя zip")
                            else:
                                name_value = tex_base or orig_key
                                name_source = "имя .tex" if tex_base else "имя zip"

                            # служебные файлы
                            new_problem_zip.writestr("name.txt", name_value)
                            new_problem_zip.writestr("index.txt", str(idx))

                        # пишем в новый problems.zip под индексом
                        new_name = f"{idx}.zip"
                        new_problems_zip.writestr(new_name, new_problem_buf.getvalue())

                        # логи по задаче
                        print(f"    ├─ Задача: {p} → {new_name}")
                        if new_main_path:
                            print(f"    │    • .tex → {new_main_path} (переименование в main.tex)")
                        else:
                            print(f"    │    • ⚠️ .tex не найден (main.tex НЕ создан)")
                        print(f"    │    • name.txt: «{name_value}» (источник: {name_source})")
                        if html_removed:
                            print(f"    │    • Удалено .html внутри задачи: {html_removed}")
                        if cleaned_cnt:
                            print(f"    │    • Очищено \\answerinput{{...}} в .tex: {cleaned_cnt}")

            # пересобираем целевой корневой архив, удаляя .html ВЕЗДЕ
            out_zip_path = os.path.join(OUTPUT_DIR, root_zip_name)
            with zipfile.ZipFile(out_zip_path, 'w', compression=zipfile.ZIP_DEFLATED) as out_zip:
                for f in root_zip.namelist():
                    if is_html(f):
                        root_html_removed += 1
                        continue
                    if f == problems_member:
                        # заменяем на новый problems.zip
                        out_zip.writestr(f, new_problems_buffer.getvalue())
                    else:
                        out_zip.writestr(f, root_zip.read(f))

    # сводный лог по архиву
    print(f"  ✓ Готово: {root_zip_name}")
    print(f"    • Задач перепаковано: {tasks_written}")
    print(f"    • Удалено .html (корень): {root_html_removed}")
    print(f"    • Удалено .html (внутри problems/задач): {inner_html_removed}")
    print(f"    • Очищено \\answerinput: {tex_cleaned}")
    print(f"    • Результат: {os.path.join(OUTPUT_DIR, root_zip_name)}")

    return (1, tasks_written, root_html_removed + inner_html_removed, tex_cleaned)

def main():
    start = time.time()
    archives_total = 0
    tasks_total = 0
    html_removed_total = 0
    tex_cleaned_total = 0

    zips = [z for z in sorted(os.listdir(SOURCE_DIR)) if z.lower().endswith(".zip")]
    if not zips:
        print("⚠️ В каталоге нет .zip архивов.")
        return

    print(f"Найдено архивов: {len(zips)}. Выходная папка: {OUTPUT_DIR}")

    for z in zips:
        a, t, h, c = process_root_zip(z)
        archives_total += a
        tasks_total += t
        html_removed_total += h
        tex_cleaned_total += c

    print("\n==================== ИТОГО ====================")
    print(f"• Обработано архивов:   {archives_total}")
    print(f"• Всего задач:          {tasks_total}")
    print(f"• Удалено .html всего:  {html_removed_total}")
    print(f"• Очищено answerinput:  {tex_cleaned_total}")
    print(f"• Время:                {time.time()-start:.2f} c")
    print(f"• Папка с результатом:  {OUTPUT_DIR}")

if __name__ == "__main__":
    main()



📦 Обработка physics__7.avgspd.zip
  • Найдено задач: 7
    • Задача 1.zip → 1.zip (name: Половинки)
    • Задача 2.zip → 2.zip (name: Катер)
    • Задача 3.zip → 3.zip (name: Плывем по течению)
    • Задача 4.zip → 4.zip (name: Смотрите под ноги)
    • Задача 5.zip → 5.zip (name: Бег по кругу)
    • Задача 6.zip → 6.zip (name: График средней скорости от пути)
    • Задача 7.zip → 7.zip (name: Бездорожье)

📦 Обработка physics__7.force.zip
  • Найдено задач: 1
    • Задача 1.zip → 1.zip (name: Перетягивание каната)

📦 Обработка physics__7arhimed.zip
  • Найдено задач: 6
    • Задача 1.zip → 1.zip (name: Шар повесили)
    • Задача 2.zip → 2.zip (name: Топим пластилином)
    • Задача 3.zip → 3.zip (name: На границе)
    • Задача 4.zip → 4.zip (name: Куб не тонет)
    • Задача 5.zip → 5.zip (name: Поплыли)
    • Задача 6.zip → 6.zip (name: Ледяные весы)

📦 Обработка physics__7friction.zip
  • Найдено задач: 4
    • Задача 1.zip → 1.zip (name: Куда везем мы кирпичи)
    • Задача 2.zip → 2.z